In [ ]:
# example: male v female cauacasian
mask_A = (X_test['SEX'] == 1) & (X_test['RAC1P'] == 1)
mask_B = (X_test['SEX'] == 2) & (X_test['RAC1P'] == 1)

shap_male = shap_values.values[mask_A.values]
shap_female = shap_values.values[mask_B.values]
feature_names = all_feature_names

shapiro_list = []
shapiro_listW = []
levenes = []
z_list = []

print(len(feature_names))

normality_warnings = 0
varience_warnings = 0

for i, feature in enumerate(feature_names):
    group1 = shap_male[:, i]
    group2 = shap_female[:, i]
    
    # Shapiro-Wilk test
    shapiro_gr1 = shapiro(group1).pvalue
    shapiro_gr2 = shapiro(group2).pvalue
    shapiroW_gr1 = shapiro(group1).statistic
    shapiroW_gr2 = shapiro(group2).statistic
    shapiro_list.append([shapiro_gr1, shapiro_gr2])
    shapiro_listW.append([shapiroW_gr1, shapiroW_gr2])

    # Levenes test ---> depends on assumption of normality!
    w_stats, p_value = levene(group1, group2, center='mean')
    levenes.append([w_stats, p_value])

    # outlier test ---> depends on assumption of normality!
    z1 = np.abs(stats.zscore(group1))
    z2 = np.abs(stats.zscore(group2))
    z_list.append([z1, z2])


print("Shapiro values: ", shapiro_list)
print("Shapiro statistic: ", shapiro_listW)
print("Levens values: ", levenes)
print("z-scores that deviate: ", z_list)

import matplotlib.pyplot as plt
import numpy as np

for i, feature in enumerate(feature_names):
    group1 = shap_male[:, i]
    group2 = shap_female[:, i]

    plt.hist(group1, bins=30, color='skyblue', edgecolor='black')

    plt.show()
    plt.hist(group2, bins=30, color='skyblue', edgecolor='black')

    plt.show()
    break